In [1]:
import importlib
import multiprocessing
import cantera
import pandas as pd
import numpy as np
import igndelay
from tqdm.auto import tqdm
from igndelay import chemkin, cantera, mechs, mp_util
import matplotlib.pyplot as plt

In [2]:
importlib.reload(chemkin)
importlib.reload(mechs)
importlib.reload(mp_util)

<module 'igndelay.mp_util' from 'e:\\Code\\selfignition-delay\\igndelay\\mp_util.py'>

In [3]:
expdata = igndelay.experimental()
mechanisms = mechs.find_mechs()
ck_env = chemkin.CKEnv(r"E:\Soft\ANSYS Inc\v172\reaction\chemkinpro.win64\bin")
ct_mechs = [(name, mechs.cantera_conv(name, mech)) for (name,mech) in tqdm(mechanisms.items(), desc='Converting mechanisms')]

Converting mechanisms: 100%|██████████| 12/12 [00:00<00:00, 1090.92it/s]


In [4]:
# poolargs = [('ct', (mech[1], expdata['T, K'][i], expdata['p, атм'][i])) for mech in ct_mechs for i in range(expdata.shape[0])]
poolargs = [(*mech, expdata['T, K'][i], expdata['p, атм'][i]) for i in range(expdata.shape[0]) for mech in ct_mechs]

In [5]:
len(poolargs)

672

In [6]:
import tracemalloc

In [7]:
tracemalloc.start()
if __name__=='__main__':
    res = []
    # for 
    with multiprocessing.Pool(6, None, 1) as pool:
        mpd = pool.map(mp_util.ct_worker, tqdm(poolargs, 'input'), 1)
        snapshot = tracemalloc.take_snapshot()
        pool.close()
        pool.join()

input: 100%|██████████| 672/672 [32:12<00:00,  2.88s/it]  


tracemalloc.start()
if __name__=='__main__':
    res = []
    # for 
    with multiprocessing.Pool(8) as pool:
        mpd = pool.map(mp_util.worker, tqdm(poolargs[:32], 'input'), 1)
        snapshot = tracemalloc.take_snapshot()
        pool.close()
        pool.join()

In [13]:
mpdict = dict(mpd)

In [41]:
results = pd.DataFrame([(p, T, mt, dt, *[round((ut-mt)/dt,2) for ut in [tc, *[mpdict[(mech, T, p)]['dT/dt']*1e6 for (mech,_) in ct_mechs]]]) for (i,(p, T, mt, dt, tc)) in expdata.iterrows()],
columns = ['p, атм', 'T, K', 't_exp', 'Δt_exp=1σ', 'Δt_calc, σ', *[f'Δ{x[0]}, σ' for x in ct_mechs]])

In [42]:
results

,"p, атм","T, K",t_exp,Δt_exp=1σ,"Δt_calc, σ","ΔAramco13, σ","ΔAramco20, σ","ΔAramco30, σ","ΔGriMech12, σ","ΔGriMech30, σ","ΔNUImech, σ","ΔRAMEC, σ","ΔSlavinskaya, σ","ΔUSCC1C3, σ","ΔZhukovC1C4, σ","ΔZhukovC1C7, σ","ΔZhukovKongCH4, σ"
0,2.54,1688.0,77.0,5.0,4.00,12.96,14.25,14.32,5.52,3.84,11.35,6.62,11.25,5.21,7.65,7.65,10.38
1,2.95,1661.0,101.0,3.0,0.00,18.75,21.19,21.37,3.88,0.70,15.90,5.75,15.14,3.34,7.48,7.48,12.20
2,3.21,1537.0,262.0,4.0,0.25,49.68,54.59,55.72,7.63,0.03,45.20,10.25,34.71,6.30,12.36,12.36,19.75
3,3.44,1584.0,189.0,20.0,-1.10,4.39,5.01,5.12,-0.24,-1.15,3.76,0.14,3.04,-0.41,0.46,0.46,1.48
4,3.76,1644.0,100.0,2.0,-2.50,25.03,28.56,28.86,3.04,-1.95,21.04,5.49,20.09,2.04,7.72,7.71,14.08
5,3.91,1487.0,343.0,4.0,3.00,39.20,39.35,39.12,15.90,2.88,39.25,18.43,39.26,12.43,20.35,20.35,28.86
6,4.24,1525.0,257.0,4.0,-5.50,39.88,44.14,45.20,2.52,-5.78,35.75,4.53,28.33,0.29,6.15,6.15,12.32
7,4.56,1715.0,61.0,10.0,-1.30,1.06,1.40,1.40,-0.67,-1.18,0.62,-0.40,0.79,-0.80,-0.16,-0.16,0.53
8,12.80,1426.0,314.0,8.0,-10.00,20.32,21.17,22.05,-0.71,-9.98,17.12,-1.04,20.89,-5.46,-0.43,-0.43,1.00
9,13.60,1570.0,89.0,5.0,-5.40,3.33,4.13,4.26,-2.62,-5.40,2.06,-2.29,4.77,-3.85,-1.90,-1.90,-0.74


In [47]:
{x: np.average(results[x]**2)**0.5 for x in results.columns[4:]}

{'Δt_calc, σ': 25.379564684435163,
 'ΔAramco13, σ': 21.07436810610056,
 'ΔAramco20, σ': 21.10836547099616,
 'ΔAramco30, σ': 21.91776106266331,
 'ΔGriMech12, σ': 16.486009219941618,
 'ΔGriMech30, σ': 8.519477997925861,
 'ΔNUImech, σ': 17.52754016642707,
 'ΔRAMEC, σ': 8.989742766063998,
 'ΔSlavinskaya, σ': 35.18813337415247,
 'ΔUSCC1C3, σ': 10.43191694135975,
 'ΔZhukovC1C4, σ': 9.28524790037555,
 'ΔZhukovC1C7, σ': 9.337258985223814,
 'ΔZhukovKongCH4, σ': 10.55704647887575}

In [52]:
{x: np.average(results[x][results['p, атм']>50]**2)**0.5 for x in results.columns[4:]}

{'Δt_calc, σ': 11.24430845927275,
 'ΔAramco13, σ': 30.56542104464395,
 'ΔAramco20, σ': 28.36088968344194,
 'ΔAramco30, σ': 30.340241204770216,
 'ΔGriMech12, σ': 34.63741854752406,
 'ΔGriMech30, σ': 11.309723089609061,
 'ΔNUImech, σ': 22.835777153007474,
 'ΔRAMEC, σ': 14.95033991708428,
 'ΔSlavinskaya, σ': 68.0096632171421,
 'ΔUSCC1C3, σ': 19.420271742317475,
 'ΔZhukovC1C4, σ': 15.07599114787844,
 'ΔZhukovC1C7, σ': 15.078648058520606,
 'ΔZhukovKongCH4, σ': 15.733176584992156}

In [54]:
ohresults = pd.DataFrame([(p, T, mt, dt, *[round((ut-mt)/dt,2) for ut in [tc, *[mpdict[(mech, T, p)]['oh']*1e6 for (mech,_) in ct_mechs]]]) for (i,(p, T, mt, dt, tc)) in expdata.iterrows()],
columns = ['p, атм', 'T, K', 't_exp', 'Δt_exp=1σ', 'Δt_calc, σ', *[f'Δ{x[0]}, σ' for x in ct_mechs]])
ohresults

,"p, атм","T, K",t_exp,Δt_exp=1σ,"Δt_calc, σ","ΔAramco13, σ","ΔAramco20, σ","ΔAramco30, σ","ΔGriMech12, σ","ΔGriMech30, σ","ΔNUImech, σ","ΔRAMEC, σ","ΔSlavinskaya, σ","ΔUSCC1C3, σ","ΔZhukovC1C4, σ","ΔZhukovC1C7, σ","ΔZhukovKongCH4, σ"
0,2.54,1688.0,77.0,5.0,4.00,15.37,16.65,16.73,8.29,6.21,13.45,9.25,12.85,7.59,10.31,10.28,13.62
1,2.95,1661.0,101.0,3.0,0.00,22.17,24.62,24.77,7.81,4.05,18.90,9.48,17.41,6.68,11.24,11.21,16.92
2,3.21,1537.0,262.0,4.0,0.25,52.31,57.23,58.36,10.61,2.56,47.53,13.10,36.47,8.84,15.20,15.22,23.77
3,3.44,1584.0,189.0,20.0,-1.10,4.85,5.48,5.58,0.28,-0.71,4.17,0.63,3.34,0.04,0.97,0.96,2.16
4,3.76,1644.0,100.0,2.0,-2.50,28.90,32.42,32.72,7.47,1.79,24.43,9.70,22.59,5.79,11.95,11.97,19.65
5,3.91,1487.0,343.0,4.0,3.00,39.26,39.46,39.37,18.35,4.97,39.39,20.77,39.29,14.55,22.70,22.71,32.53
6,4.24,1525.0,257.0,4.0,-5.50,41.78,46.04,47.11,4.67,-3.96,37.43,6.56,29.57,2.11,8.20,8.21,15.45
7,4.56,1715.0,61.0,10.0,-1.30,1.62,1.95,1.96,-0.02,-0.63,1.11,0.22,1.14,-0.25,0.47,0.47,1.34
8,12.80,1426.0,314.0,8.0,-10.00,20.62,21.48,22.35,-0.38,-9.68,17.39,-0.71,21.03,-5.17,-0.10,-0.10,1.13
9,13.60,1570.0,89.0,5.0,-5.40,3.69,4.49,4.62,-2.21,-5.06,2.38,-1.89,4.91,-3.51,-1.50,-1.50,0.19
